# 🛠️ Advanced Google ADK: Mastering Tools
## Practical Tutorial for Google Colab

---

## 📋 Index
1. **Initial Setup**
2. **What are Tools in ADK?**
3. **Pre-built Tools**
4. **Creating Custom Tools**
5. **Best Practices**
6. **Advanced Use Cases**

## 1. Initial Setup

### ADK Installation

In [ ]:
# Install Google ADK
!pip install -q google-adk==1.4.2

In [ ]:
# Necessary imports
import os
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from getpass import getpass

#### Option 1: Enter them directly

In [ ]:
# Request API Key securely
if 'GOOGLE_API_KEY' not in os.environ:
    print("🔑 Please enter your Google API Key:")
    api_key = getpass("API Key: ")
    os.environ['GOOGLE_API_KEY'] = api_key
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
    print("\n✅ API Key configured correctly")
else:
    print("✅ API Key already configured")

# Verify that variables are configured
print(f"\n📋 Configured environment variables:")
print(f"   - GOOGLE_API_KEY: {'✓' if os.environ.get('GOOGLE_API_KEY') else '✗'}")
print(f"   - GOOGLE_GENAI_USE_VERTEXAI: {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI', 'Not configured')}")

print("✅ ADK installed and configured correctly!")

#### Option 2: Load via Dotenv

In [ ]:
from dotenv import load_dotenv
# Load environment variables from .env if it exists
load_dotenv(override=True)

## 2. What are Tools in ADK?

**Tools** are capabilities or functions that an agent can invoke to:
- 🔍 Search for information
- 🧮 Perform calculations
- 🌐 Interact with APIs
- 📊 Process data
- ⚙️ Execute specific actions

### Key Concepts

In [ ]:
# Visualize the tools concept
print("""
🤖 LLM AGENT
    ↓
📦 TOOLBOX
    ├── 🔍 Google Search
    ├── 🧮 Calculator
    ├── 📊 Database
    └── 🌐 External APIs
    ↓
🌍 ACTION IN THE REAL WORLD
""")

## 3. Pre-built Tools

### Example: GoogleSearchTool

##### Our inference function

In [ ]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User query: {query}")

    # Prepare the user message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "The agent did not produce a final response." # Default value

    # Key concept: run_async executes the agent's logic and generates events.
    # We iterate through events to find the final response.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key concept: is_final_response() marks the message that concludes the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assume the text response is in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle possible errors/escalations
                final_response_text = f"The agent escalated: {event.error_message or 'No specific message.'}"
            # Add more validations here if needed (e.g., specific error codes)
            break # Stop processing events once final response is found

    print(f"<<< Agent response: {final_response_text}")

In [ ]:
# Create an agent with Google search tool
search_agent = LlmAgent(
    name="GoogleResearcher",
    model="gemini-2.5-flash",
    description="An agent that uses Google search to answer current questions",
    tools=[google_search],  # Pre-built tool
    instruction=(
        "You are an expert researcher. "
        "Use Google search to find up-to-date information. "
        "Cite your sources when possible."
    )
)

print("✅ Researcher agent created with GoogleSearchTool")

#### Test the researcher agent

In [ ]:
# Key concept: SessionService stores conversation history and state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants to identify the interaction context
APP_NAME = "search_tool_agent"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will occur
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: This is the main component that manages interaction with the agent.
runner = Runner(agent=search_agent,
                app_name=APP_NAME,
                session_service=session_service)


In [ ]:
await call_agent_async("What is the name of the new pope for 2025?",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)

### Example: BuiltInCodeExecutor for code execution

In [ ]:
from google.adk.code_executors import BuiltInCodeExecutor

AGENT_NAME = "calculator_agent"
APP_NAME = "calculator"
USER_ID = "user1234"
SESSION_ID = "session_code_exec_async"
GEMINI_MODEL = "gemini-2.0-flash"

code_agent = LlmAgent(
    name=AGENT_NAME,
    model=GEMINI_MODEL,
    code_executor=BuiltInCodeExecutor(),  # enable code execution
    instruction="""You are a calculator agent.
    When provided with a mathematical expression, write and execute Python code to calculate the result.
    Return only the final numerical result as plain text, without markdown formatting or code blocks.
    """,
    description="Executes Python code to perform calculations.",
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)
runner = Runner(agent=code_agent, app_name=APP_NAME, session_service=session_service)

In [ ]:
# Asynchronous function to send a query to the agent and process the events it returns
async def call_agent_async_code(query, runner, user_id, session_id):
    # Create the message with the user's text in the format required by ADK
    content = types.Content(role="user", parts=[types.Part(text=query)])
    print(f"\n--- Executing Query: {query} ---")

    # Variable to store the agent's final response
    final_response_text = "No final text response was captured."

    # Start the asynchronous loop to process each event emitted by the agent
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=content
    ):
        print(f"Event ID: {event.id}, Author: {event.author}")

        # --- Check if the event contains specific parts like code or results ---
        has_specific_part = False
        if event.content and event.content.parts:
            for part in event.content.parts:  # Iterate through all content parts
                if part.executable_code:
                    # If there's executable code generated by the agent, print it
                    print(
                        f"  Debug: Code generated by agent:\n```python\n{part.executable_code.code}\n```"
                    )
                    has_specific_part = True
                elif part.code_execution_result:
                    # If there's code execution result, show the result
                    print(
                        f"  Debug: Code execution result: {part.code_execution_result.outcome} - Output:\n{part.code_execution_result.output}"
                    )
                    has_specific_part = True
                elif part.text and not part.text.isspace():
                    # If there's plain text, show it (not considered a "specific" part)
                    print(f"  Text: '{part.text.strip()}'")
                    # Don't mark `has_specific_part = True` here to not interfere with final response logic

        # --- Check if it's a final response (after handling specific parts) ---
        # Only consider this response as final if there were no specific parts before
        if not has_specific_part and event.is_final_response():
            if (
                event.content
                and event.content.parts
                and event.content.parts[0].text
            ):
                final_response_text = event.content.parts[0].text.strip()
                print(f"==> Final Agent Response: {final_response_text}")
            else:
                print("==> Final Agent Response: [No text content in final event]")

    # Final closing message
    print("-" * 30)

#### Test 1

In [ ]:
await call_agent_async_code("Calculate the value of ((5 + 7 + 10) * 12) to the power of 2", runner, USER_ID, SESSION_ID)

#### Test 2

In [ ]:
await call_agent_async_code("What is the factorial of 10?", runner, USER_ID, SESSION_ID)

## 4. Creating Custom Tools

### 🧮 Tool 1: Simple Calculator

In [ ]:
def add_numbers(a: int, b: int) -> dict:
    """
    Adds two integers and returns the result in a structured format.

    Use this tool when you need to perform addition of two integer values.
    
    Args:
        a (int): First number to add.
        b (int): Second number to add.

    Returns:
        dict: A dictionary with the following keys:
            - 'status' (str): "success" if the operation was successful.
            - 'result' (int): Result of the addition (a + b).
            - 'operation' (str): Description of the operation performed.
    """
    try:
        print(f"🧮 add_numbers tool called with a={a}, b={b}")
        result = a + b
        return {
            "status": "success",
            "result": result,
            "operation": f"Addition of {a} + {b}"
        }
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"An error occurred while adding the numbers: {str(e)}"
        }

### 🛍️ Tool 2: Product Search

In [ ]:
import json

def search_product_by_name(product_name: str) -> dict:
    """
    Searches for a product by its name in the catalog and returns a dictionary with its details.

    Use this tool if the user requests information about a specific product.

    Args:
        product_name (str): Name of the product to search for (case insensitive).

    Returns:
        dict: Dictionary with the following possible fields:
            - 'status' (str): "success" if product was found, "error" if not.
            - 'product' (dict, optional): Product details if found.
            - 'error_message' (str, optional): Explanatory message if product not found.
    """
    print(f"🛍️ Searching for product: '{product_name}'")

    # Database simulation
    products_db = {
        "gaming laptop": {
            "id": "LPG001",
            "name": "Gaming Laptop Pro",
            "price": 1500,
            "stock": 10,
            "features": ["RTX 4070", "32GB RAM", "1TB SSD"]
        },
        "mechanical keyboard": {
            "id": "TEC005",
            "name": "Mechanical RGB Keyboard",
            "price": 120,
            "stock": 25,
            "features": ["Cherry MX Switches", "RGB", "TKL"]
        },
        "4k monitor": {
            "id": "MON003",
            "name": "4K HDR Monitor",
            "price": 400,
            "stock": 5,
            "features": ["27 inches", "144Hz", "HDR10"]
        }
    }

    product = products_db.get(product_name.lower())
    
    if product:
        return {
            "status": "success",
            "product": product
        }
    else:
        return {
            "status": "error",
            "error_message": f"Product '{product_name}' not found in catalog."
        }

### 📈 Tool 3: Percentage Calculation

In [ ]:
def calculate_percentage(value: float, percentage: float) -> dict:
    """
    Calculates the percentage of a base value.

    Use this tool to calculate discounts, increases, commissions, proportions or other values relative to a percentage.

    Args:
        value (float): Base value on which to calculate the percentage.
        percentage (float): Percentage to apply (e.g. 15 for 15%).

    Returns:
        dict: Dictionary with the following keys:
            - 'status' (str): "success" if the calculation was successful.
            - 'result' (float): Calculated value, rounded to two decimal places.
            - 'details' (str): Explanation of the calculation performed.
    """
    try:
        print(f"📈 Calculating {percentage}% of {value}")
        result = round((value * percentage) / 100, 2)
        return {
            "status": "success",
            "result": result,
            "details": f"{percentage}% of {value} is {result}"
        }
    except Exception as e:
        return {
            "status": "error",
            "error_message": f"Could not calculate percentage: {str(e)}"
        }

### Create an agent with custom tools

In [ ]:
# Create agent with our custom tools
function_tool_agent = LlmAgent(
    name="FunctionAgent",
    model="gemini-2.0-flash",
    description="A useful agent that uses custom tools for calculations and searches",
    tools=[add_numbers, calculate_percentage, search_product_by_name],  # Our tools
    generate_content_config=types.GenerateContentConfig(
        temperature=0.1,  # Low temperature for precise calculations
        max_output_tokens=300
    ),
    instruction=(
        "You are a precise assistant. "
        "Help the user using the available tools. "
    )
)

print("✅ Calculator agent created with custom tools")

In [ ]:
# Key concept: SessionService stores conversation history and state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants to identify the interaction context
APP_NAME = "function_tool_agent"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will occur
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: This is the main component that manages interaction with the agent.
runner = Runner(agent=function_tool_agent,
                app_name=APP_NAME,
                session_service=session_service)


### Test the function agent

In [ ]:
await call_agent_async("How much is 15% of 1200?", runner=runner, user_id=USER_ID, session_id=SESSION_ID)

In [ ]:
await call_agent_async("What are the features and price of the mechanical keyboard?", runner=runner, user_id=USER_ID, session_id=SESSION_ID)

In [ ]:
await call_agent_async("How much is 10 + 9991?", runner=runner, user_id=USER_ID, session_id=SESSION_ID)

## 5. Best Practices


### Importance of Type Hints and Docstrings

In [ ]:
# Example of POORLY defined tool (DON'T DO THIS)

def poorly_defined_tool(x, y):  # ❌ No type hints
    """Does something"""  # ❌ Vague description, no useful context for the agent
    return x + y  # ❌ Unknown what types of values it expects or what it actually does


# Example of WELL defined tool (DO THIS)

from typing import Dict

def convert_temperature(celsius: float) -> Dict[str, float]:
    """
    Converts a temperature in Celsius to Fahrenheit and Kelvin.

    Use this tool when the user needs to transform a temperature from the Celsius scale
    to other common scales like Fahrenheit or Kelvin.

    Args:
        celsius (float): Temperature in degrees Celsius.

    Returns:
        dict: Dictionary with the following keys:
            - 'celsius': Original value in Celsius.
            - 'fahrenheit': Value converted to Fahrenheit.
            - 'kelvin': Value converted to Kelvin.
    """
    fahrenheit = (celsius * 9/5) + 32
    kelvin = celsius + 273.15

    return {
        "status": "success",
        "celsius": celsius,
        "fahrenheit": round(fahrenheit, 2),
        "kelvin": round(kelvin, 2)
    }

print("✅ Example of well-defined tool created")

## 6. Advanced Use Cases {#advanced-cases}

### 🎯 Tool with State and Context

In [ ]:
shopping_cart: list[dict] = []  # Simulated cart state

def add_to_cart(product: str, quantity: int) -> dict:
    """
    Adds a valid product to the shopping cart.

    Use this tool when the user wants to add a specific product with a certain quantity.

    Args:
        product (str): Name of the product to add (case insensitive).
        quantity (int): Desired quantity of the product.

    Returns:
        dict: Dictionary with:
            - 'status': "success" or "error"
            - 'message': Confirmation or reason for failure
            - 'current_cart_size': Total number of items in cart after action (if successful)
    """
    global shopping_cart

    shopping_cart.append({
        "product": product.lower(),
        "quantity": quantity
    })

    total_items = sum(item["quantity"] for item in shopping_cart)

    return {
        "status": "success",
        "message": f"Added {quantity}x '{product}' to cart.",
        "current_cart_size": total_items
    }

def view_cart() -> dict:
    """
    Returns the current contents of the shopping cart.

    Use this tool if the user requests to review what products they have added to the cart.

    Returns:
        dict: Dictionary with:
            - 'status': "success" or "empty"
            - 'items': List of products in cart (if any)
            - 'total_items': Total number of units (if any)
            - 'message': Brief text about cart status
    """
    global shopping_cart

    if not shopping_cart:
        return {
            "status": "empty",
            "items": [],
            "total_items": 0,
            "message": "The cart is empty."
        }

    total_items = sum(item["quantity"] for item in shopping_cart)

    return {
        "status": "success",
        "items": shopping_cart,
        "total_items": total_items,
        "message": f"There are {len(shopping_cart)} products in cart, totaling {total_items} units."
    }

### Create e-commerce agent

In [ ]:
# Specialized e-commerce agent
ecommerce_agent = LlmAgent(
    name="EcommerceAgent",
    model="gemini-2.5-flash",
    description="Online shopping assistant",
    tools=[
        search_product_by_name,
        add_to_cart,
        view_cart,
        calculate_percentage  # For discounts
    ],
    generate_content_config=types.GenerateContentConfig(
        temperature=0.2,
        max_output_tokens=400
    ),
    instruction=(
        "You are a friendly and helpful shopping assistant. "
        "Help users find products, add them to cart "
        "and calculate discounts when needed. "
        "Be proactive in suggesting related products."
    )
)

print("✅ E-commerce agent created")

### Test shopping flow

In [ ]:
# Key concept: SessionService stores conversation history and state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants to identify the interaction context
APP_NAME = "ecommerce_agent"
USER_ID = "user_4"
SESSION_ID = "004" # Using a fixed ID for simplicity

# Create the specific session where the conversation will occur
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: This is the main component that manages interaction with the agent.
runner = Runner(agent=ecommerce_agent,
                app_name=APP_NAME,
                session_service=session_service)


In [ ]:
# Simulate a shopping flow
shopping_flow = [
    "Show me information about the gaming laptop",
    "Add 1 Gaming Laptop Pro to cart",
    "I also want 2 Mechanical Gaming Keyboards",
    "What's in my cart?",
    "How much would it be with a 10% discount on 1500?"
]

print("🛍️ Simulating shopping flow:\n")

for step in shopping_flow:
    print("-" * 60 +"\n")
    await call_agent_async(step, runner=runner, user_id=USER_ID, session_id=SESSION_ID)

In [ ]:
shopping_cart

## 🎯 Final Exercise: Create Your Own Tool

In [ ]:
# EXERCISE: Complete this tool
def analyze_sentiment(text: str) -> Dict[str, any]:
    """
    TODO: Complete this tool that analyzes the sentiment of a text.
    
    Args:
        text: Text to analyze
        
    Returns:
        Dictionary with:
        - sentiment: "positive", "negative" or "neutral"
        - confidence: float between 0 and 1
        - keywords: list of words that indicate sentiment
    """
    # YOUR CODE HERE
    # Hint: You can use lists of positive/negative words
    # or implement simple logic based on keywords
    
    pass

### 📚 Additional Resources

In [ ]:
resources = {
    "ADK Documentation": "https://github.com/google/genkit",
    "Gemini API": "https://ai.google.dev",
    "Best Practices": "https://cloud.google.com/apis/design",
    "Community": "https://groups.google.com/g/google-ai-developer-community"
}

print("📚 RESOURCES FOR FURTHER LEARNING:")
for name, url in resources.items():
    print(f"  • {name}: {url}")

## 🎉 Congratulations!

You have completed the tools tutorial in Google ADK. Now you have the knowledge to:
- ✅ Use pre-built tools
- ✅ Create custom tools with proper definitions
- ✅ Integrate multiple tools in an agent
- ✅ Implement validations and state management
- ✅ Debug and optimize your tools

### 🚀 Next Steps

In [ ]:
print("""
🎯 CHALLENGES TO PRACTICE:

1. Create a tool that interacts with a real API
2. Implement a tool system for a support chatbot
3. Design tools that work together to solve complex problems
4. Experiment with tools that maintain state between calls
5. Create a specialized agent for your domain of interest

Share your creations with the community! 🌟
""")

---

**Created with ❤️ for the ADK developer community**